In [22]:
import pandas as pd
import os
import pymongo as pm
import re
pd.options.display.max_rows = 200

In [23]:
workbook_path = os.path.join('DFS', 'Resources', 'DraftKingsNoNotes.xlsx')
workbook = pd.ExcelFile(workbook_path)


In [24]:
vegas_dash = workbook.parse('Vegas Dash')
top_guns = workbook.parse('TOP GUNS')
wr_dash = workbook.parse('WR Dash')
te_dash = workbook.parse('TE Dash')
rb_dash = workbook.parse('RB Dash')
qb_dash = workbook.parse('QB Dash')
dst_dash = workbook.parse('DST Dash')

In [25]:
client = pm.MongoClient('localhost', 27017)

db = client.test_database
SS_Data = db['Saber_Sim_Data']
_4f4_Proj = db['_4for4_Projection_Data']
_4f4_Ceil = db['_4for4_Ceiling_Data']
_4f4_WR_cb = db['_4for4_WR_cb']
_4f4_RedZ = db['_4for4__RedZone_Data']
_4f4_WR_fp = db['_4for4_WR_fantasy_points']
_4f4_RB_fp = db['_4for4_RB_fantasy_points']
_4f4_RB_tar = db['_4for4_RB_target']
airy_WR = db['AirY_WR']
airy_TE = db['AirY_TE']
ETR_Ceil = db['ETR_Ceiling']

In [26]:
def position_names(pos, db):
    data = []

    for collection in db.list_collection_names():
        col = db[collection]
        positions = ['Pos', 'Position', 'position']
        for fil in positions:
            doc_count = col.count_documents(filter={fil : pos})
            if doc_count > 0:
                cursor = col.find({fil : pos},{'_id':False})
                for i in range(doc_count):
                    data.append(cursor[i])
    names = []
    for row in data:
        if 'full_name' in row:
            name = row['full_name']
            if not any(word in name for word in names):
                names.append(name)
        elif 'Name' in row:
            name = row['Name']
            if not any(word in name for word in names):
                names.append(name)
        elif 'Player' in row:
            name = row['Player']
            if not any(word in name for word in names):
                names.append(name)
    return names

In [37]:
def player_query(player, db):
    player_info = {'name' : player}
    
    for collection in db.list_collection_names():
        col = db[collection]
        name_forms = ['Name', 'Player', 'full_name']
        for nm in name_forms:
            doc_count = col.count_documents(filter={nm : player})
            if doc_count > 0:
                cursor = col.find({nm : player}, {'_id' : False, nm : False})
                for i in range(doc_count):
                    player_info.update(cursor[i])
                    
    return player_info


In [42]:
qbs =  position_names('QB', db)

In [43]:
qb_data = [player_query(player, db) for player in qbs]